In [1]:
from IPython.core.display import HTML
import requests
import time
import pandas as pd
import datetime
import pytz

In [2]:
import os
import numpy as np
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/master project/data/META' #เข้า drive

Mounted at /content/drive


In [3]:
api_key = 'J4AM5YKF88UA41WO'

In [4]:
def _get_data(symbols, time_from, time_to, api_key):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={symbols}&time_from={time_from}&time_to={time_to}&limit=1000&apikey={api_key}"
    r = requests.get(url)
    data = r.json()
    return data

In [5]:
def get_dataset(time_from="20190101T0000",
                time_to="20250301T0000",
                MAX_API_CALLS_PER_DAY = 99999, # Free tier only allows 25 API calls per day
                MAX_API_CALLS_PER_MIN = 300 # Free tier only allows 5 api calls per minute
               ):
    data_list=[]
    for i in range(1,MAX_API_CALLS_PER_DAY+1):
        if i%600==0:
            time.sleep(65)

        data=_get_data('META',time_from,time_to,api_key)
        if 'feed' not in data:break
        if len(data['feed'])==0: break
        data_list.append(data)
        time_to=data['feed'][-1]['time_published'][:-2] # Take all the way up to last 2 since api only takes minute level granularity
    df=pd.concat([pd.DataFrame(data['feed']) for data in data_list])
    # Extract META specific relevance (we didn't use it in video)

    # Only take tickers with META in headline
    df=df[df.title.str.contains('META|Facebook|Instagram',case=False)]
    # Only take when # of tickers = 1
    # Drop duplicates..
    df.drop_duplicates(subset=['summary'],inplace=True,keep='first')
    # Set index to time published
    #df.set_index('time_published',inplace=True)
    # Sort by time published
    #df.sort_index(inplace=True)
    return df


In [6]:
df = get_dataset()

In [7]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.tz_localize('UTC')

In [8]:
df['time_published']=df['time_published'].dt.tz_convert('US/Eastern')

In [9]:
df['time_published'] = pd.to_datetime(df['time_published']).dt.strftime('%Y-%m-%d')

In [10]:
df

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
8,Trump company files trademarks for branded met...,https://cointelegraph.com/news/trump-company-f...,2025-02-28,[Christopher Tepedino],"DTTM Operations, a Trump-owned company, has fi...",https://s3.cointelegraph.com/uploads/2025-02/0...,Cointelegraph,n/a,cointelegraph.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.100093,Neutral,"[{'ticker': 'GOOG', 'relevance_score': '0.0891..."
19,Meta Platforms ( META ) Down 4.2% Since Last...,https://www.zacks.com/stock/news/2423778/meta-...,2025-02-28,[Zacks Equity Research],Meta Platforms (META) reported earnings 30 day...,https://staticx-tuner.zacks.com/images/default...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Earnings', 'relevance_score': '0.7...",0.129191,Neutral,"[{'ticker': 'META', 'relevance_score': '0.5611..."
24,Understanding Meta Platforms's Position In Int...,https://www.benzinga.com/insights/news/25/02/4...,2025-02-28,[Benzinga Insights],Amidst the fast-paced and highly competitive b...,https://www.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.9...",0.350680,Bullish,"[{'ticker': 'META', 'relevance_score': '0.4756..."
49,"Activist Investor ValueAct, Once Linked To Ste...",https://www.benzinga.com/markets/equities/25/0...,2025-02-28,[Kaustubh Bagalkote],"Activist investor ValueAct Holdings, L.P., kno...",https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",0.288768,Somewhat-Bullish,"[{'ticker': 'MSFT', 'relevance_score': '0.3056..."
55,Mark Zuckerberg's Meta Wants To Do An AI App B...,https://www.benzinga.com/markets/equities/25/0...,2025-02-27,[Kaustubh Bagalkote],OpenAI CEO Sam Altman teased the possibility o...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Earnings', 'relevance_score': '0.1...",0.296324,Somewhat-Bullish,"[{'ticker': 'GOOG', 'relevance_score': '0.3175..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,"Metaverse Land Just Sold For $4,309 ( 5,000 S...",https://www.benzinga.com/markets/cryptocurrenc...,2022-06-13,[Benzinga Insights],What happened: Virtual land tokenized as an NF...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Technology', 'relevance_score': '1...",-0.012154,Neutral,"[{'ticker': 'META', 'relevance_score': '0.1379..."
719,"Why Amazon, Apple, and Meta Platforms Are Fall...",https://www.fool.com/investing/2022/06/13/why-...,2022-06-13,[Chris Neiger],Investors really didn't like last week's infla...,https://g.foolcdn.com/image/?url=https%3A%2F%2...,Motley Fool,n/a,www.fool.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.297630,Somewhat-Bearish,"[{'ticker': 'META', 'relevance_score': '0.1572..."
4,What Are Whales Doing With Meta Platforms,https://www.benzinga.com/markets/options/22/06...,2022-06-13,[Benzinga Insights],A whale with a lot of money to spend has taken...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,Markets,www.benzinga.com,"[{'topic': 'Financial Markets', 'relevance_sco...",-0.020044,Neutral,"[{'ticker': 'GS', 'relevance_score': '0.126251..."
5,Meta Platforms ( META ) Halts Dual-Camera Sm...,https://www.zacks.com/stock/news/1938334/meta-...,2022-06-13,[Zacks Investment Research],Meta Platforms (META) has halted the developme...,https://staticx-tuner.zacks.com/images/article...,Zacks Commentary,n/a,www.zacks.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",-0.281084,Somewhat-Bearish,"[{'ticker': 'MSFT', 'relevance_score': '0.1603..."


In [11]:
News_META = df[["time_published", "title", "summary","overall_sentiment_score"]]

In [12]:
News_META

,time_published,title,summary,overall_sentiment_score
8,2025-02-28,Trump company files trademarks for branded met...,"DTTM Operations, a Trump-owned company, has fi...",0.100093
19,2025-02-28,Meta Platforms ( META ) Down 4.2% Since Last...,Meta Platforms (META) reported earnings 30 day...,0.129191
24,2025-02-28,Understanding Meta Platforms's Position In Int...,Amidst the fast-paced and highly competitive b...,0.350680
49,2025-02-28,"Activist Investor ValueAct, Once Linked To Ste...","Activist investor ValueAct Holdings, L.P., kno...",0.288768
55,2025-02-27,Mark Zuckerberg's Meta Wants To Do An AI App B...,OpenAI CEO Sam Altman teased the possibility o...,0.296324
...,...,...,...,...
718,2022-06-13,"Metaverse Land Just Sold For $4,309 ( 5,000 S...",What happened: Virtual land tokenized as an NF...,-0.012154
719,2022-06-13,"Why Amazon, Apple, and Meta Platforms Are Fall...",Investors really didn't like last week's infla...,-0.297630
4,2022-06-13,What Are Whales Doing With Meta Platforms,A whale with a lot of money to spend has taken...,-0.020044
5,2022-06-13,Meta Platforms ( META ) Halts Dual-Camera Sm...,Meta Platforms (META) has halted the developme...,-0.281084


In [13]:
!pip install nltk

In [14]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [15]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [16]:
sia = SentimentIntensityAnalyzer()
# Analyze sentiment for each summary

x = str(News_META['summary']).encode("utf-8")
News_META['sentiment_score'] = News_META['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])

<ipython-input-16-ba63a2c0eb56>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  News_META['sentiment_score'] = News_META['summary'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [17]:
News_META

,time_published,title,summary,overall_sentiment_score,sentiment_score
8,2025-02-28,Trump company files trademarks for branded met...,"DTTM Operations, a Trump-owned company, has fi...",0.100093,0.0000
19,2025-02-28,Meta Platforms ( META ) Down 4.2% Since Last...,Meta Platforms (META) reported earnings 30 day...,0.129191,0.0000
24,2025-02-28,Understanding Meta Platforms's Position In Int...,Amidst the fast-paced and highly competitive b...,0.350680,0.6590
49,2025-02-28,"Activist Investor ValueAct, Once Linked To Ste...","Activist investor ValueAct Holdings, L.P., kno...",0.288768,0.4588
55,2025-02-27,Mark Zuckerberg's Meta Wants To Do An AI App B...,OpenAI CEO Sam Altman teased the possibility o...,0.296324,0.2263
...,...,...,...,...,...
718,2022-06-13,"Metaverse Land Just Sold For $4,309 ( 5,000 S...",What happened: Virtual land tokenized as an NF...,-0.012154,0.0000
719,2022-06-13,"Why Amazon, Apple, and Meta Platforms Are Fall...",Investors really didn't like last week's infla...,-0.297630,-0.3374
4,2022-06-13,What Are Whales Doing With Meta Platforms,A whale with a lot of money to spend has taken...,-0.020044,-0.2023
5,2022-06-13,Meta Platforms ( META ) Halts Dual-Camera Sm...,Meta Platforms (META) has halted the developme...,-0.281084,0.0000


In [18]:
News_META_Nozero=News_META.loc[News_META['sentiment_score'] != 0]
News_META_Nozero

,time_published,title,summary,overall_sentiment_score,sentiment_score
24,2025-02-28,Understanding Meta Platforms's Position In Int...,Amidst the fast-paced and highly competitive b...,0.350680,0.6590
49,2025-02-28,"Activist Investor ValueAct, Once Linked To Ste...","Activist investor ValueAct Holdings, L.P., kno...",0.288768,0.4588
55,2025-02-27,Mark Zuckerberg's Meta Wants To Do An AI App B...,OpenAI CEO Sam Altman teased the possibility o...,0.296324,0.2263
67,2025-02-27,Apollo Global Management In Talks To Lead $35 ...,Apollo is negotiating a $35 billion deal to he...,0.322508,0.8442
81,2025-02-27,Assessing Meta Platforms's Performance Against...,In the dynamic and fiercely competitive busine...,0.294453,0.6908
...,...,...,...,...,...
672,2022-06-14,"Facebook, Twitter, Google Set To Face Hefty EU...",The updated European Union code of practice is...,-0.110156,-0.4767
687,2022-06-14,EU tackles deepfakes with pressure on Big Tech...,The European Commission is expected to update ...,-0.178451,-0.4767
719,2022-06-13,"Why Amazon, Apple, and Meta Platforms Are Fall...",Investors really didn't like last week's infla...,-0.297630,-0.3374
4,2022-06-13,What Are Whales Doing With Meta Platforms,A whale with a lot of money to spend has taken...,-0.020044,-0.2023


In [19]:
daily_avg_sentiment_META = News_META.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_META

,time_published,sentiment_score
0,2022-06-12,0.177900
1,2022-06-13,-0.053970
2,2022-06-14,-0.092589
3,2022-06-15,0.115163
4,2022-06-16,0.296262
...,...,...
942,2025-02-24,0.278183
943,2025-02-25,0.072850
944,2025-02-26,0.614340
945,2025-02-27,0.411460


In [20]:
daily_avg_sentiment_Nozero_META = News_META_Nozero.groupby('time_published')['sentiment_score'].mean().reset_index()
daily_avg_sentiment_Nozero_META

,time_published,sentiment_score
0,2022-06-12,0.177900
1,2022-06-13,-0.269850
2,2022-06-14,-0.208325
3,2022-06-15,0.460650
4,2022-06-16,0.296262
...,...,...
912,2025-02-24,0.417275
913,2025-02-25,0.072850
914,2025-02-26,0.614340
915,2025-02-27,0.514325


In [21]:
daily_avg_sentiment_META.to_csv(os.path.join(path,'daily_avg_sentiment_META.csv'))
News_META.to_csv(os.path.join(path,'News_META.csv'))

In [22]:
daily_avg_sentiment_Nozero_META.to_csv(os.path.join(path,'daily_avg_sentiment_Nozero_META.csv'))
News_META_Nozero.to_csv(os.path.join(path,'News_META_Nozero.csv'))